## Supervised Learning using BERT for Sentiment Analysis and Notation Prediction

### Data Extraction and Preprocessing (we follow the same process as in data_cleaning but we keep a different sample of the data)

In [1]:
import pandas as pd
import nltk
from cleantext import clean
import tqdm
import numpy as np
from deep_translator import GoogleTranslator

Since the GPL-licensed package `unidecode` is not installed, using Python's `unicodedata` package which yields worse results.


In [2]:
reviews_df = pd.read_csv("data/reviews.csv", sep=";")
reviews_df.head(3)

,category_link,company_link,company_name,score,description,review_page_nb,review_link,review_score,review_title,review_text,category
0,https://fr.trustpilot.com/categories/food_beve...,https://fr.trustpilot.com/review/lefourgon.com,Le Fourgon,"4,9",Le Fourgon vous livre vos boissons consignées ...,329,https://fr.trustpilot.com/reviews/65a5388a60d6...,5,Application conviviale pour passer ses…,Application conviviale pour passer ses command...,food_beverages_tobacco
1,https://fr.trustpilot.com/categories/food_beve...,https://fr.trustpilot.com/review/lefourgon.com,Le Fourgon,"4,9",Le Fourgon vous livre vos boissons consignées ...,329,https://fr.trustpilot.com/reviews/65a53245a223...,5,Très facile pour la commande,"Très facile pour la commande, très rapide et l...",food_beverages_tobacco
2,https://fr.trustpilot.com/categories/food_beve...,https://fr.trustpilot.com/review/lefourgon.com,Le Fourgon,"4,9",Le Fourgon vous livre vos boissons consignées ...,329,https://fr.trustpilot.com/reviews/659f0e15dd57...,5,Première expérience réussie !,"Pour nous, c'était une première.Ravis d'avoir ...",food_beverages_tobacco


In [3]:
reviews_df = reviews_df[["company_name", "score", "review_score", "review_title", "review_text", "category"]]

In [4]:
# we merge the title and the text of the review
reviews_df["review"] = reviews_df["review_title"] + " " + reviews_df["review_text"]
reviews_df.head(3)

,company_name,score,review_score,review_title,review_text,category,review
0,Le Fourgon,"4,9",5,Application conviviale pour passer ses…,Application conviviale pour passer ses command...,food_beverages_tobacco,Application conviviale pour passer ses… Applic...
1,Le Fourgon,"4,9",5,Très facile pour la commande,"Très facile pour la commande, très rapide et l...",food_beverages_tobacco,Très facile pour la commande Très facile pour ...
2,Le Fourgon,"4,9",5,Première expérience réussie !,"Pour nous, c'était une première.Ravis d'avoir ...",food_beverages_tobacco,"Première expérience réussie ! Pour nous, c'éta..."


In [5]:
clean(reviews_df["review"].values[0], no_emoji=True, no_line_breaks=True, lower=False).replace("#", "")

'Application conviviale pour passer ses Application conviviale pour passer ses commandes. Manquent juste les ingredients pas evidents a trouver.Notification par SMS peu de temps avant le creneau choisi pour un creneau precis de 20 minutes.Livreurs agreables.Merci le Fourgon !'

In [6]:
reviews_df["review"] = reviews_df["review"].apply(lambda x: clean(x, no_emoji=True, no_line_breaks=True, lower=False).replace("#", ""))

In [7]:
reviews_df["review"].values[100]

"Exceptionnel La brulerie Belleville est devenue mon unique fournisseur en cafe, le cafe est vraiment d'une qualite exceptionnelle ! De plus, le service apres-vente est d'une qualite humaine devenue bien rare a present !"

#### We Keep only a sample of the data (the top 10 companies with the most reviews) Because of the traduction cost of the "review" column

In [8]:
# number of reviews per company
reviews_df["company_name"].value_counts()[:10]

company_name
Hardloop           236
Alltricks          210
Ekosport           140
vertbaudet         140
Mode Tactique      120
Weenect            120
AAAEP              120
Handball Store     111
Meyclub            100
Bleen              100
Name: count, dtype: int64

In [9]:
# we keep only the reviews of the first 10 companies in terms of number of reviews
reviews_df = reviews_df[reviews_df["company_name"].isin(reviews_df["company_name"].value_counts()[:10].index)]
reviews_df["company_name"].value_counts().sum()

1397

Translation